In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import geocoder
import matplotlib.pyplot as plt

In [2]:
sensor = pd.read_csv("../project-data/ParkingSensorData.csv")

/anaconda/envs/fastai/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train = pd.read_csv("./cleaned_train.csv")
test = pd.read_csv("./cleaned_test.csv")

In [4]:
sensor.head().transpose()

,0,1,2,3,4
BLOCK_ID,41522,41524,50001,50002,10903
STREET_NAME,FILLMORE ST,FILLMORE ST,JACKSON ST,JACKSON ST,09TH AVE
BLOCK_NUM,22,24,1,2,3
STREET_BLOCK,FILLMORE ST 2200,FILLMORE ST 2400,JACKSON ST 100,JACKSON ST 200,09TH AVE 300
AREA_TYPE,Pilot,Pilot,Pilot,Pilot,Control
PM_DISTRICT_NAME,Fillmore,Fillmore,Downtown,Downtown,Inner Richmond
RATE,NaN,NaN,NaN,NaN,NaN
START_TIME_DT,01-jul-2012 21:00:00,01-jul-2012 22:00:00,01-jul-2012 02:00:00,01-jul-2012 08:00:00,01-jul-2012 08:00:00
TOTAL_TIME,61200,64800,79200,82800,93600
TOTAL_OCCUPIED_TIME,35847,27576,39985,30465,85690


In [5]:
sensor.shape

(7902291, 33)

In [6]:
#get unique, lowercase values
train_st = train.Street.unique()
train_st = [x.lower() for x in train_st]

In [7]:
sensor_st = sensor.STREET_NAME.unique()[:-1] #dont get nan
sensor_st2 = [x.lower() for x in sensor_st ]
sensor_st2 = ['geary st' if x == 'geary blvd' else x for x in sensor_st2]

In [8]:
#change street names to match each other
sensor_st3 = []
for x in sensor_st2:
    if x.endswith("st"):
        sensor_st3.append(x+"reet") 
    elif x.endswith("ave"):
        sensor_st3.append(x+"nue")
    else:
        sensor_st3.append(x)
sensor_st3

['fillmore street',
 'jackson street',
 '09th avenue',
 '10th avenue',
 '14th avenue',
 'berry street',
 '01st street',
 'california street',
 'chestnut street',
 'folsom street',
 'jones street',
 '02nd street',
 'franklin street',
 'market street',
 '03rd street',
 'clay street',
 'fremont street',
 'front street',
 '04th street',
 'geary street',
 'mason street',
 'clement street',
 'kearny street',
 '18th street',
 '22nd street',
 'leavenworth street',
 '23rd street',
 'battery street',
 'davis street',
 'golden gate avenue',
 'mcallister street',
 'bay street',
 'divisadero street',
 'drumm street',
 'gough street',
 'grove street',
 'laguna street',
 'beach street',
 'fell street',
 'greenwich street',
 'hawthorne street',
 'lombard street',
 'mallorca way',
 'beale street',
 'brannan street',
 'hayes street',
 'bryant street',
 'howard street',
 'filbert street',
 'hoff street',
 'buchanan street',
 'bush street',
 'jessie street',
 'montgomery street',
 'larkin street',
 'king 

In [9]:
#make dict of old and new sensor names
st_dict = dict(zip(sensor_st, sensor_st3))

In [10]:
#match check (should be 0)
[x for x in train_st if x not in sensor_st3]

[]

In [11]:
sensor['STREET_NAME'].value_counts()

FILLMORE ST          402738
GEARY BLVD           286608
CLEMENT ST           204720
VALENCIA ST          184248
MONTGOMERY ST        163776
WASHINGTON ST        163776
02ND ST              157248
SACRAMENTO ST        143304
BEACH ST             143304
SANSOME ST           143304
JACKSON ST           141792
CLAY ST              141792
NORTH POINT ST       134214
MISSION ST           123676
EMBARCADERO SOUTH    122832
WEBSTER ST           122832
UNION ST             122832
VAN NESS AVE         122815
JEFFERSON ST         122798
PINE ST              108754
HAYES ST             107640
CHESTNUT ST          102360
POLK ST              102360
FRANKLIN ST          102360
BRANNAN ST           102360
03RD ST              102360
LOMBARD ST           102360
POST ST               96401
KEARNY ST             93144
SPEAR ST              85120
                      ...  
TEHAMA ST             20472
NATOMA ST             20472
REDWOOD ST            20472
AVILA ST              20472
20TH ST             

In [12]:
# replace sensor street name with new vals
sensor['STREET_NAME'] = sensor['STREET_NAME'].map(st_dict)

In [13]:
sensor.STREET_NAME.value_counts()

fillmore street       402738
geary street          286608
clement street        204720
valencia street       184248
washington street     163776
montgomery street     163776
02nd street           157248
sansome street        143304
beach street          143304
sacramento street     143304
clay street           141792
jackson street        141792
north point street    134214
mission street        123676
embarcadero south     122832
webster street        122832
union street          122832
van ness avenue       122815
jefferson street      122798
pine street           108754
hayes street          107640
lombard street        102360
franklin street       102360
chestnut street       102360
polk street           102360
03rd street           102360
brannan street        102360
post street            96401
kearny street          93144
spear street           85120
                       ...  
beale street           20472
merchant street        20472
redwood street         20472
divisadero str

In [14]:
sensor.isnull().sum()

BLOCK_ID                     0
STREET_NAME                  1
BLOCK_NUM                    1
STREET_BLOCK                 1
AREA_TYPE                    1
PM_DISTRICT_NAME             1
RATE                   5242511
START_TIME_DT                1
TOTAL_TIME                   1
TOTAL_OCCUPIED_TIME          1
TOTAL_VACANT_TIME            1
TOTAL_UNKNOWN_TIME           1
OP_TIME                      1
OP_OCCUPIED_TIME             1
OP_VACANT_TIME               1
OP_UNKNOWN_TIME              1
NONOP_TIME                   1
NONOP_OCCUPIED_TIME          1
NONOP_VACANT_TIME            1
NONOP_UNKNOWN_TIME           1
GMP_TIME                     1
GMP_OCCUPIED_TIME            1
GMP_VACANT_TIME              1
GMP_UNKNOWN_TIME             1
COMM_TIME                    1
COMM_OCCUPIED_TIME           1
COMM_VACANT_TIME             1
COMM_UNKNOWN_TIME            1
CAL_MONTH_NAME               1
CAL_YEAR                     1
CAL_DATE                     1
DAY_TYPE                     1
TIME_OF_

In [15]:
#the last row contains all na - drop those
sensor = sensor[:-1]

In [16]:
sensor.tail()

,BLOCK_ID,STREET_NAME,BLOCK_NUM,STREET_BLOCK,AREA_TYPE,PM_DISTRICT_NAME,RATE,START_TIME_DT,TOTAL_TIME,TOTAL_OCCUPIED_TIME,...,GMP_UNKNOWN_TIME,COMM_TIME,COMM_OCCUPIED_TIME,COMM_VACANT_TIME,COMM_UNKNOWN_TIME,CAL_MONTH_NAME,CAL_YEAR,CAL_DATE,DAY_TYPE,TIME_OF_DAY
7902285,33429,buchanan street,29.0,BUCHANAN ST 2900,Control,Union,NaN,13-oct-2011 14:00:00,32400.0,30775.0,...,0.0,0.0,0.0,0.0,0.0,October,2011.0,13-oct-2011,weekday,1400.0
7902286,33418,buchanan street,18.0,BUCHANAN ST 1800,Pilot,Fillmore,NaN,16-oct-2011 09:00:00,43200.0,35311.0,...,0.0,0.0,0.0,0.0,0.0,October,2011.0,16-oct-2011,weekend,900.0
7902287,33003,brannan street,3.0,BRANNAN ST 300,Pilot,South Embarcadero,NaN,20-oct-2011 19:00:00,201600.0,189141.0,...,0.0,0.0,0.0,0.0,0.0,October,2011.0,20-oct-2011,weekday,1900.0
7902288,33103,bryant street,3.0,BRYANT ST 300,Pilot,South Embarcadero,NaN,30-oct-2011 08:00:00,122400.0,11331.0,...,0.0,0.0,0.0,0.0,0.0,October,2011.0,30-oct-2011,weekend,800.0
7902289,33103,bryant street,3.0,BRYANT ST 300,Pilot,South Embarcadero,NaN,30-oct-2011 11:00:00,122400.0,25912.0,...,0.0,0.0,0.0,0.0,0.0,October,2011.0,30-oct-2011,weekend,1100.0


In [17]:
#only get rows in sensor data that are in train
#sensor2 = sensor[sensor['STREET_NAME'].isin(train_st)]
sensor2 = sensor

In [18]:
print(sensor.shape)
print(sensor2.shape)

(7902290, 33)
(7902290, 33)


In [19]:
sensor2.head(4)

,BLOCK_ID,STREET_NAME,BLOCK_NUM,STREET_BLOCK,AREA_TYPE,PM_DISTRICT_NAME,RATE,START_TIME_DT,TOTAL_TIME,TOTAL_OCCUPIED_TIME,...,GMP_UNKNOWN_TIME,COMM_TIME,COMM_OCCUPIED_TIME,COMM_VACANT_TIME,COMM_UNKNOWN_TIME,CAL_MONTH_NAME,CAL_YEAR,CAL_DATE,DAY_TYPE,TIME_OF_DAY
0,41522,fillmore street,22.0,FILLMORE ST 2200,Pilot,Fillmore,NaN,01-jul-2012 21:00:00,61200.0,35847.0,...,0.0,0.0,0.0,0.0,0.0,July,2012.0,01-jul-2012,weekend,2100.0
1,41524,fillmore street,24.0,FILLMORE ST 2400,Pilot,Fillmore,NaN,01-jul-2012 22:00:00,64800.0,27576.0,...,0.0,0.0,0.0,0.0,0.0,July,2012.0,01-jul-2012,weekend,2200.0
2,50001,jackson street,1.0,JACKSON ST 100,Pilot,Downtown,NaN,01-jul-2012 02:00:00,79200.0,39985.0,...,0.0,0.0,0.0,0.0,0.0,July,2012.0,01-jul-2012,weekend,200.0
3,50002,jackson street,2.0,JACKSON ST 200,Pilot,Downtown,NaN,01-jul-2012 08:00:00,82800.0,30465.0,...,0.0,0.0,0.0,0.0,0.0,July,2012.0,01-jul-2012,weekend,800.0


In [20]:
len(sensor2['STREET_BLOCK'].unique())

407

## Add lat an long to senor data

In [21]:
#add SF on end
sensor2["STREET_BLOCK_renamed"] = sensor2["STREET_BLOCK"] + ", San Francisco, CA" 

In [22]:
#build dictionaries of lat/long for street intersections
block_unique = sensor2["STREET_BLOCK_renamed"].unique()

dblock = dict(zip(block_unique, pd.Series(block_unique).apply(geocoder.google).apply(lambda x: x.latlng)))
dblock = {k: v for k, v in dblock.items() if v is not None}
print(f'There are {len(block_unique)} uniqueblocks')
print(f'dict has {len(dblock)} entries')

Status code Unknown from https://maps.googleapis.com/maps/api/geocode/json: ERROR - HTTPSConnectionPool(host='maps.googleapis.com', port=443): Max retries exceeded with url: /maps/api/geocode/json?address=VALENCIA+ST+700%2C+San+Francisco%2C+CA&bounds=&components=&region=&language= (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x13d9756a0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))
Status code Unknown from https://maps.googleapis.com/maps/api/geocode/json: ERROR - HTTPSConnectionPool(host='maps.googleapis.com', port=443): Max retries exceeded with url: /maps/api/geocode/json?address=SANSOME+ST+0%2C+San+Francisco%2C+CA&bounds=&components=&region=&language= (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x13d975908>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))
Status code Unknown from https://maps.googleapi

Error: [('system library', 'fopen', 'Too many open files'), ('BIO routines', 'BIO_new_file', 'system lib'), ('x509 certificate routines', 'X509_load_cert_crl_file', 'system lib')]

In [23]:
#map dictionaries to data to get lat/long
sensor2['coord_block'] = sensor2["STREET_BLOCK_renamed"].map(dblock)

In [24]:
#find still empties in street_from and rerun
stillempty = sensor2[sensor2['coord_block'].isnull()]['STREET_BLOCK_renamed'].unique()

In [25]:
print(len(stillempty))

279


In [26]:
new_from = {}
still_to_find = []
for name in stillempty:
    g = geocoder.google(name)
    if g.latlng is not None:
        new_from[name] = g.latlng
    else:
        still_to_find.append(name)

In [ ]:
#combine dicts
dblock2 = {**dblock, **new_from}
print(len(new_from))
print(len(dblock2))

187
315


In [ ]:
new_from = {}
still_to_find2 = []
for name in still_to_find:
    g = geocoder.google(name)
    if g.latlng is not None:
        new_from[name] = g.latlng
    else:
        still_to_find2.append(name)

In [ ]:
#combine dicts
dblock3 = {**dblock2, **new_from}
print(len(new_from))
print(len(dblock3))

In [ ]:
#create dictionary for each coordinate
dblock_lat = dict(zip(dblock3.keys(),[dblock3[k][0] for k in dblock3]))
dblock_long = dict(zip(dblock3.keys(),[dblock3[k][1] for k in dblock3]))
print(len(dblock_lat))
print(len(dblock_long))


In [ ]:
#map dictionaries to data to get lat/long
sensor2['block_lat'] = sensor2['STREET_BLOCK_renamed'].map(dblock_lat)
sensor2['block_long'] = sensor2['STREET_BLOCK_renamed'].map(dblock_long)


In [ ]:
sensor2.head(10)

### clean up variables

In [54]:
sensor2.rename(index=str, columns={"STREET_NAME": "Street"}, inplace=True)

In [56]:
sensor2['Date'] = pd.to_datetime(sensor2['CAL_DATE'], infer_datetime_format=True)
sensor2['hour'] = sensor2['TIME_OF_DAY'].apply(lambda x: x/100).astype(int)
sensor2['month'] = pd.to_datetime(sensor2['Date']).dt.month
sensor2['dayofweek'] = pd.to_datetime(sensor2['Date']).dt.weekday
sensor2['is_weekend'] = ((pd.DatetimeIndex(sensor2['Date']).dayofweek) // 5 == 1).astype(int)

KeyboardInterrupt: 

In [ ]:
sensor2['TOTAL_VAC_RATIO'] = sensor2['TOTAL_VACANT_TIME']/sensor2['TOTAL_TIME']

In [ ]:
#drop unecessary variables
sensor2 = sensor2.drop(['Date', 'START_TIME_DT', 'CAL_DATE', 'DAY_TYPE','STREET_BLOCK_renamed', 'coord_block'], axis = 1)

### export to csv

In [50]:
sensor2.to_csv("./geocoded_sensor.csv")